<a href="https://colab.research.google.com/github/argan719/DataScience/blob/main/val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Data Test

In [ ]:
import os, json, numpy as np
from matplotlib import pyplot as plt
from ultralytics import YOLO
from PIL import Image

MODEL_NAME = "yolov8n11"
OPEN_DATASETS = ("CVC-ClinicDB", "CVC-ColonDB", "ETIS-LaribPolypDB")

model = YOLO(f"yolov8/runs/train/{MODEL_NAME}/weights/last.pt")  # load a pretrained model (recommended for training)

for ds in OPEN_DATASETS:
    model.val(
        data=f"/mnt/Disk3/snu_project/opendata/{ds}/yolo-data.yaml",
        name=f"{ds}-{MODEL_NAME}",
        device=5,
        exist_ok=True,
        batch=40,
        save_json=True,
    )

    with open(f'runs/detect/{ds}-{MODEL_NAME}/predictions.json') as f:
        pred = json.load(f)

    bboxes = {}
    for d in pred:
        image_id = str(d['image_id'])
        x, y, w, h = d['bbox']
        score = d['score']
        if score <= 0:
            continue
        if image_id not in bboxes:
            bboxes[image_id] = []
        bboxes[image_id].append(((x, y, x + w, y + h), score))

    def calculate_iou(bbox1, bbox2):
        x11, y11, x12, y12 = bbox1
        x21, y21, x22, y22 = bbox2
        inter = max(min(x12, x22) - max(x11, x21), 0) * max(min(y12, y22) - max(y11, y21), 0)
        union = (x12 - x11) * (y12 - y11) + (x22 - x21) * (y22 - y21) - inter
        return inter / max(union, 1e-12)

    polyps = {}

    for image in os.listdir(f'/mnt/Disk3/snu_project/opendata/{ds}/yolo-format'):
        if image[-4:] not in ('.png', '.bmp'): continue
        with Image.open(f'/mnt/Disk3/snu_project/opendata/{ds}/yolo-format/{image}') as im:
            width, height = im.size
        with open(f'/mnt/Disk3/snu_project/opendata/{ds}/yolo-format/{image[:-4]}.txt') as f:
            for line in f.readlines():
                try:
                    _, x, y, w, h = map(float, line.split())
                    true = ((x - w / 2) * width, (y - h / 2) * height, (x + w / 2) * width, (y + h / 2) * height)
                    k = len(polyps)
                    polyps[k] = []
                    for bbox, score in bboxes.get(image[:-4], []):
                        iou = calculate_iou(bbox, true)
                        # print(image, true, bbox, iou)
                        if iou > 0:
                            polyps[k].append((iou, score))
                except Exception as e:
                    print(image, e, line)

    for k, v in polyps.items():
        v.sort()
        vv = []
        for x, y in v:
            while vv and vv[-1][1] <= y:
                vv.pop()
            vv.append((x, y))
        polyps[k] = vv

    print(f'Polyps: {len(polyps)}, Max positive: {sum(len(v) > 0 for v in polyps.values())}')
    print(sum(any(iou > 0.25 and score > 0.2 for iou, score in v) for v in polyps.values()) / len(polyps))

    xs, ys = np.meshgrid(
        np.linspace(1, 0, 101, False),
        np.linspace(1, 0, 101, False),
    )
    zs = np.ndarray(xs.shape)
    for i in range(zs.shape[0]):
        for j in range(zs.shape[1]):
            iou_thres = xs[i][j]
            conf_thres = ys[i][j]
            cnt = 0
            for v in polyps.values():
                cnt += any(iou > iou_thres and score > conf_thres for iou, score in v)
            zs[i][j] = cnt / len(polyps)

    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(projection='3d')
    ax.set_title(f'Open Data ({ds})')
    ax.set_xlabel('IoU threshold')
    ax.set_ylabel('Confidence threshold')
    ax.set_zlabel('Sensitivity')
    ax.set_xlim(0, 1)
    ax.set_ylim(1, 0)
    ax.set_zlim(0, 1)
    ax.plot_surface(xs, ys, zs, cmap='plasma', rstride=1, cstride=1, antialiased=False, linewidth=0)
    ax.plot_wireframe(xs, ys, zs, cstride=5, rstride=5, color='black')
    # ax.scatter(xs, ys, zs, c=zs, cmap='viridis', marker='.', alpha=0.5)
    fig.show()
    fig.savefig(f'sensitivity_{ds}.png')

Ultralytics YOLOv8.0.107 🚀 Python-3.10.10 torch-2.0.0 CUDA:5 (NVIDIA GeForce RTX 3070, 7982MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /mnt/Disk3/snu_project/opendata/CVC-ClinicDB/yolo-format.cache... 612 images, 0 backgrounds, 0 corrupt: 100%|██████████| 612/612 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:07<00:00,  2.07it/s]
                   all        612        612      0.657      0.593      0.601      0.182
Speed: 1.0ms preprocess, 3.3ms inference, 0.0ms loss, 1.9ms postprocess per image
Saving runs/detect/CVC-ClinicDB-yolov8n11/predictions.json...
Results saved to runs/detect/CVC-ClinicDB-yolov8n11


[337.411, 250.791, 299.154, 261.771]
Polyps: 612, Max positive: 581
0.8006535947712419


Ultralytics YOLOv8.0.107 🚀 Python-3.10.10 torch-2.0.0 CUDA:5 (NVIDIA GeForce RTX 3070, 7982MiB)
val: Scanning /mnt/Disk3/snu_project/opendata/CVC-ColonDB/yolo-format.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:06<00:00,  1.29it/s]
                   all        300        300      0.705      0.653      0.652      0.175
Speed: 2.2ms preprocess, 4.6ms inference, 0.0ms loss, 2.0ms postprocess per image
Saving runs/detect/CVC-ColonDB-yolov8n11/predictions.json...
Results saved to runs/detect/CVC-ColonDB-yolov8n11


[533.409, 448.205, 439.408, 500.0]
Polyps: 300, Max positive: 299
0.9466666666666667


Ultralytics YOLOv8.0.107 🚀 Python-3.10.10 torch-2.0.0 CUDA:5 (NVIDIA GeForce RTX 3070, 7982MiB)
val: Scanning /mnt/Disk3/snu_project/opendata/ETIS-LaribPolypDB/yolo-format.cache... 196 images, 0 backgrounds, 0 corrupt: 100%|██████████| 196/196 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:07<00:00,  1.57s/it]
                   all        196        196      0.507       0.49      0.439      0.124
Speed: 3.7ms preprocess, 5.4ms inference, 0.0ms loss, 1.8ms postprocess per image
Saving runs/detect/ETIS-LaribPolypDB-yolov8n11/predictions.json...
Results saved to runs/detect/ETIS-LaribPolypDB-yolov8n11


[1156.923, 911.163, 864.397, 780.67]
Polyps: 196, Max positive: 189
0.7908163265306123


### Test Data Test

In [ ]:
from ultralytics import YOLO

MODEL_NAME = "yolov8n12"
model = YOLO(f"yolov8/runs/train/{MODEL_NAME}/weights/last.pt")  # load a pretrained model (recommended for training)

model.val(
    data="testset-640x640/data.yaml",
    device=5,
    exist_ok=True,
    batch=40,
    save_json=True,
)


Ultralytics YOLOv8.0.107 🚀 Python-3.10.10 torch-2.0.0 CUDA:5 (NVIDIA GeForce RTX 3070, 7982MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /mnt/Disk3/snu_project/jeehak/testset-640x640/labels.cache... 20050 images, 6051 backgrounds, 0 corrupt: 100%|██████████| 20050/20050 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 502/502 [02:53<00:00,  2.89it/s] 
                   all      20050      14236      0.577      0.457      0.418      0.129
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Saving runs/detect/val/predictions.json...
Results saved to runs/detect/val


ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x7fc6724c11e0>
fitness: 0.15775537004406678
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.12884])
names: {0: 'polyp'}
plot: True
results_dict: {'metrics/precision(B)': 0.5773938135468589, 'metrics/recall(B)': 0.45701039617870187, 'metrics/mAP50(B)': 0.4179685072821725, 'metrics/mAP50-95(B)': 0.1288427992398328, 'fitness': 0.15775537004406678}
save_dir: PosixPath('runs/detect/val')
speed: {'preprocess': 0.18504429339173428, 'inference': 1.5862106028340404, 'loss': 0.000549254572005046, 'postprocess': 1.0529442856139375}

In [ ]:
import os, json, cv2, numpy as np
from matplotlib import pyplot as plt

with open('runs/detect/val/predictions.json') as f:
    pred = json.load(f)

bboxes = {}
for d in pred:
    image_id = d['image_id']
    x, y, w, h = d['bbox']
    score = d['score']
    if score <= 0:
        continue
    if image_id not in bboxes:
        bboxes[image_id] = []
    bboxes[image_id].append(((x, y, x + w, y + h), score))

def calculate_iou(bbox1, bbox2):
    x11, y11, x12, y12 = bbox1
    x21, y21, x22, y22 = bbox2
    inter = max(min(x12, x22) - max(x11, x21), 0) * max(min(y12, y22) - max(y11, y21), 0)
    union = (x12 - x11) * (y12 - y11) + (x22 - x21) * (y22 - y21) - inter
    return inter / max(union, 1e-12)

polyps = {}
sizes = {}

for image in os.listdir('testset-640x640/images'):
    video = image.rsplit('-', 1)[0]
    if video not in sizes:
        vid = cv2.VideoCapture(f'/mnt/Disk3/snu_project/testdata/videos/{video.replace("-", "/")}.mp4')
        height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
        width = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
        scale = 640 / max(height, width)
        sizes[video] = scale
    scale = sizes[video]
    label = f'/mnt/Disk3/snu_project/testdata/videos/{image[:-4].replace("-", "/")}.txt'
    with open(label) as f:
        for line in f.readlines():
            try:
                x1, y1, x2, y2, pid = map(int, line.split(',')[:5])
                if pid == 0 or pid == 9: continue
                true = (x1 * scale, y1 * scale, x2 * scale, y2 * scale)
                k = (video, pid)
                if k not in polyps:
                    polyps[k] = []
                for bbox, score in bboxes.get(image[:-4], []):
                    iou = calculate_iou(bbox, true)
                    if iou > 0:
                        polyps[k].append((iou, score))
            except Exception as e:
                print(label, e, line)

for k, v in polyps.items():
    v.sort()
    vv = []
    for x, y in v:
        while vv and vv[-1][1] <= y:
            vv.pop()
        vv.append((x, y))
    polyps[k] = vv

print(f'Polyps: {len(polyps)}, Max positive: {sum(len(v) > 0 for v in polyps.values())}')
print(sum(any(iou > 0.5 and score > 0.3 for iou, score in v) for v in polyps.values()) / len(polyps))

xs, ys = np.meshgrid(
    np.linspace(1, 0, 101, False),
    np.linspace(1, 0, 101, False),
)
zs = np.ndarray(xs.shape)
for i in range(zs.shape[0]):
    for j in range(zs.shape[1]):
        iou_thres = xs[i][j]
        conf_thres = ys[i][j]
        cnt = 0
        for v in polyps.values():
            cnt += any(iou > iou_thres and score > conf_thres for iou, score in v)
        zs[i][j] = cnt / len(polyps)

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection='3d')
# ax.set_title('Test Data')
ax.set_xlabel('IoU threshold')
ax.set_ylabel('Confidence threshold')
ax.set_zlabel('Polyp sensitivity')
ax.set_xlim(0, 1)
ax.set_ylim(1, 0)
ax.set_zlim(0, 1)
ax.plot_surface(xs, ys, zs - 0.005, cmap='plasma', rstride=1, cstride=1, antialiased=False, linewidth=0)
ax.plot_wireframe(xs, ys, zs, cstride=5, rstride=5, color='black')
fig.show()
fig.savefig('polyp_sensitivity.png')

Polyps: 45, Max positive: 45
1.0


### Export YOLOv8 Model

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8/runs/train/yolov8n12/weights/last.pt")  # load a pretrained model (recommended for training)

model.export(
    format='tflite',
    imgsz=640,
    optimize=True,
    int8=True,
    simplify=True,
    nms=True,
)

Ultralytics YOLOv8.0.107 🚀 Python-3.10.10 torch-2.0.0 CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from yolov8/runs/train/yolov8n12/weights/last.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (34.9 MB)

TensorFlow SavedModel: starting export with tensorflow 2.12.0...

ONNX: starting export with onnx 1.13.1 opset 17...
ONNX: simplifying with onnxsim 0.4.24...


================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 9.0s, saved as yolov8/runs/train/yolov8n12/weights/last.onnx (11.6 MB)

TensorFlow SavedModel: running 'onnx2tf -i yolov8/runs/train/yolov8n12/weights/last.onnx -o yolov8/runs/train/yolov8n12/weights/last_saved_model -nuo --non_verbose -oiqt -qt per-tensor'
fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8
TensorFlow SavedModel: export success ✅ 253.0s, saved as yolov8/runs/train/yolov8n12/weights/last_saved_model (39.1 MB)

TensorFlow Lite: starting export with tensorflow 2.12.0...
TensorFlow Lite: export success ✅ 0.0s, saved as yolov8/runs/train/yolov8n12/weights/last_saved_model/last_int8.tflite (3.0 MB)

Export complete (255.7s)
Results saved to /mnt/Disk3/snu_project/jeehak/yolov8/runs/train/yolov8n12/weights
Predict:         yolo predict task=detect model=yolov8/runs/train/yolov8n12/weights/last_saved_model/last_in

'yolov8/runs/train/yolov8n12/weights/last_saved_model/last_int8.tflite'

: 